In [1]:
import os
import numpy as np
import cv2
import tkinter as tk
from tkinter import filedialog
from tqdm import tqdm
import albumentations as A
from datetime import datetime
import time

def create_augmentation_pipeline():
    """Create augmentation pipeline with 15 different transformations"""
    
    # List of different augmentations to apply
    augmentations = [
        # Geometric transformations
        A.Compose([
            A.HorizontalFlip(p=1.0),
            A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=1.0)
        ]),
        A.Compose([
            A.VerticalFlip(p=1.0),
            A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=1.0)
        ]),
        A.Compose([
            A.Rotate(limit=45, p=1.0),
            A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=1.0)
        ]),
        A.Compose([
            A.Rotate(limit=90, p=1.0),
            A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=1.0)
        ]),
        A.Compose([
            A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=45, p=1.0),
            A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=1.0)
        ]),
        
        # Intensity transformations
        A.Compose([
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=1.0)
        ]),
        A.Compose([
            A.RandomGamma(gamma_limit=(80, 120), p=1.0)
        ]),
        A.Compose([
            A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), p=1.0)
        ]),
        
        # Noise and blur
        A.Compose([
            A.GaussNoise(var_limit=(10.0, 50.0), p=1.0)
        ]),
        A.Compose([
            A.GaussianBlur(blur_limit=(3, 7), p=1.0)
        ]),
        
        # Combinations
        A.Compose([
            A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=1.0),
            A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=1.0)
        ]),
        A.Compose([
            A.GridDistortion(num_steps=5, distort_limit=0.3, p=1.0),
            A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=1.0)
        ]),
        A.Compose([
            A.OpticalDistortion(distort_limit=0.2, shift_limit=0.15, p=1.0),
            A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=1.0)
        ]),
        A.Compose([
            A.RandomCrop(height=3840, width=3840, p=1.0),  # Slight crop
            A.Resize(height=4096, width=4096, p=1.0),      # Resize back to original
            A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=1.0)
        ]),
        A.Compose([
            A.RandomRotate90(p=1.0),
            A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15, p=1.0),
            A.GaussNoise(var_limit=(5.0, 30.0), p=0.5)
        ])
    ]
    
    return augmentations

def augment_images(input_dir, output_dir, num_augmentations=15):
    """Augment TEM images and masks"""
    # Create augmentation pipeline
    augmentations = create_augmentation_pipeline()
    
    # Get all TIF files and their corresponding masks
    all_files = sorted(os.listdir(input_dir))
    image_files = [f for f in all_files if f.endswith('.tif')]
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Iterate through all images
    print(f"Found {len(image_files)} original images")
    print(f"Generating {num_augmentations} augmentations for each image")
    
    for image_file in tqdm(image_files, desc="Processing Images"):
        # Extract image name without extension
        image_name = os.path.splitext(image_file)[0]
        
        # Check if mask exists
        mask_file = f"{image_name}_mask.png"
        if mask_file not in all_files:
            print(f"Warning: Mask not found for {image_file}, skipping")
            continue
        
        # Read image and mask
        image_path = os.path.join(input_dir, image_file)
        mask_path = os.path.join(input_dir, mask_file)
        
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        
        if image is None or mask is None:
            print(f"Error reading image or mask: {image_file}")
            continue
        
        # Save original image and mask to output directory
        cv2.imwrite(os.path.join(output_dir, image_file), image)
        cv2.imwrite(os.path.join(output_dir, mask_file), mask)
        
        # Create augmentations
        for aug_idx in range(num_augmentations):
            # Select augmentation (cycling through the list or random)
            transform = augmentations[aug_idx % len(augmentations)]
            
            # Apply augmentation
            augmented = transform(image=image, mask=mask)
            aug_image = augmented['image']
            aug_mask = augmented['mask']
            
            # Create augmented filenames
            aug_image_filename = f"{image_name}_aug_{aug_idx+1}.tif"
            aug_mask_filename = f"{image_name}_aug_{aug_idx+1}_mask.png"
            
            # Save augmented image and mask
            cv2.imwrite(os.path.join(output_dir, aug_image_filename), aug_image)
            cv2.imwrite(os.path.join(output_dir, aug_mask_filename), aug_mask)
    
    print(f"Augmentation complete. Files saved to: {output_dir}")

def get_input_directory():
    """Get input directory using multiple methods"""
    input_dir = None
    
    # Method 1: Try using tkinter dialog
    try:
        root = tk.Tk()
        root.withdraw()
        
        print("Waiting for folder selection dialog to appear...")
        print("If no dialog appears, check your taskbar or behind other windows.")
        print("Dialog will timeout in 10 seconds if not used.")
        
        # Force window to front
        root.attributes('-topmost', True)
        root.update()
        
        # Start timer for dialog
        start_time = time.time()
        input_dir = None
        
        # Give the dialog 10 seconds to be used
        while time.time() - start_time < 10 and input_dir is None:
            input_dir = filedialog.askdirectory(title="Select Directory with Original TEM Images and Masks")
            root.update_idletasks()
            root.update()
            time.sleep(0.1)
            
            # Break if directory selected
            if input_dir:
                break
    except Exception as e:
        print(f"GUI dialog failed: {e}")
    
    # Method 2: If GUI failed or timed out, use console input
    if not input_dir:
        print("\nFolder dialog failed or timed out.")
        print("Please enter the full path to your images directory:")
        input_dir = input("Path: ").strip()
    
    # Verify directory exists
    if not os.path.exists(input_dir):
        print(f"Error: Directory {input_dir} does not exist.")
        return None
    
    return input_dir

def main():
    print("TEM Image Augmentation Tool")
    print("===========================")
    
    # Get input directory
    input_dir = get_input_directory()
    
    if not input_dir:
        print("No valid directory provided. Exiting.")
        return
    
    print(f"Selected directory: {input_dir}")
    
    # Create output directory with timestamp
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_dir = os.path.join(os.path.dirname(input_dir), f'augmented_images_{timestamp}')
    
    # Set number of augmentations
    num_augmentations = 15
    print(f"Will create {num_augmentations} augmentations per image")
    
    # Perform augmentation
    augment_images(input_dir, output_dir, num_augmentations)

if __name__ == '__main__':
    main()

TEM Image Augmentation Tool
Waiting for folder selection dialog to appear...
If no dialog appears, check your taskbar or behind other windows.
Dialog will timeout in 10 seconds if not used.


C:\Users\mans4777\AppData\Roaming\Python\Python39\site-packages\albumentations\core\validation.py:58: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
C:\Users\mans4777\AppData\Local\Temp\ipykernel_20340\1680542117.py:51: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0), p=1.0)
C:\Users\mans4777\AppData\Local\Temp\ipykernel_20340\1680542117.py:59: UserWarning: Argument(s) 'alpha_affine' are not valid for transform ElasticTransform
  A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=1.0),
C:\Users\mans4777\AppData\Local\Temp\ipykernel_20340\1680542117.py:67: UserWarning: Argument(s) 'shift_limit' are not valid for transform OpticalDistortion
  A.OpticalDistortion(distort_limit=0.2, shift_limit=0.15, p=1.0),
C:\Users\mans4777\AppData\Local\Temp\ipykernel_20340\1680542117.py:78: UserWarning: Argument(s) 'var_limit' are n

Selected directory: D:/Rajat/RAW+ masked images
Will create 15 augmentations per image
Found 20 original images
Generating 15 augmentations for each image


Processing Images: 100%|██████████| 20/20 [02:08<00:00,  6.45s/it]

Augmentation complete. Files saved to: D:/Rajat\augmented_images_20250228_173006
